# Performance Comparison

In [ ]:
from shapely.geometry import Polygon
# Agent config
num_agents = 500
agent_radius = 0.1  

# Map config
map_type = "swarm" # Available maps: empty, corridor, obstacle, cross
num_samples = 100 # Available num_samples: 100, 200, 300, 400, 500

# config starts and goals

start_regions = [Polygon([[0, 160], [0, 0], [50, 0], [50, 160]])]
goal_regions = [Polygon([[150, 160], [150, 0], [200, 0], [200, 160]])]

num_starts = 5
num_goals = 5


In [ ]:
# Load map and instance
import os
import pickle

import numpy as np 

from swarm_prm.utils.agent_assignment import get_agent_assignment

map_fname = "{}_{}.pkl".format(map_type, num_samples)
fname = os.path.join("../maps", map_fname)
with open(fname, "rb") as f:
    gaussian_prm = pickle.load(f) 

# Randomly choose starts and goals from the candidates, and assign them a weight
# Make sure all agents can fit in the start and goal configuration

candid_starts_idx = gaussian_prm.get_node_index(start_regions)
candid_goals_idx = gaussian_prm.get_node_index(goal_regions)

starts_idx = np.random.choice(candid_starts_idx, num_starts)
goals_idx = np.random.choice(candid_goals_idx, num_goals)

# Weights for the Gaussian components in the start and goal config

starts_weight = np.random.rand(num_starts)
starts_weight /= np.sum(starts_weight)

goals_weight = np.random.rand(num_goals)
goals_weight /= np.sum(goals_weight)

# starts_weight = [0.5, 0.5]
# goals_weight = [0.2, 0.8]

starts_agent_count = get_agent_assignment(num_agents, starts_weight)
goals_agent_count = get_agent_assignment(num_agents, goals_weight)

print(starts_idx, goals_idx)
print(starts_agent_count, goals_agent_count)

In [ ]:
from swarm_prm.solvers.macro import SOLVER_REGISTRY

# solver_cls = SOLVER_REGISTRY["FormationControlSolver"]
solver_cls = SOLVER_REGISTRY["TEGSolver"]

solver = solver_cls(gaussian_prm, agent_radius, 
              starts_agent_count=starts_agent_count, goals_agent_count=goals_agent_count,
              starts_idx=starts_idx, goals_idx=goals_idx,
              num_agents=num_agents, time_limit=60)
solution = solver.solve()
assert solution["success"], "solver failed."
timestep = solution["timestep"]
paths = solution["paths"]
g_nodes = solution["g_nodes"]
candid_starts_idx = solution["starts_idx"]
candid_goals_idx = solution["goals_idx"]

In [ ]:
from matplotlib import pyplot as plt
from swarm_prm.utils import paths_to_macro
from swarm_prm.solvers.micro import EvaluationSolver

macro_solution = paths_to_macro(paths)

timestep = max(macro_solution.keys())

solver = EvaluationSolver(g_nodes, macro_solution, agent_radius, timestep, 
                                     num_agents, 
                                     candid_starts_idx,
                                     candid_goals_idx,
                                     starts_agent_count,
                                     goals_agent_count
                                     )

single_agent_paths, cost = solver.solve()
fig, ax = gaussian_prm.visualize_map()

for start in starts_idx:
    gaussian_prm.gaussian_nodes[start].visualize(ax)

for goal in goals_idx:
    gaussian_prm.gaussian_nodes[goal].visualize(ax, edgecolor="b")

cmap = plt.get_cmap("rainbow")
colors = [cmap(i / num_agents) for i in range(num_agents)]

for i, path in enumerate(single_agent_paths):
    x_coords = [loc[0] for loc in path]
    y_coords = [loc[1] for loc in path]
    ax.plot(x_coords, y_coords, '-', label='Path', color=colors[i], linewidth=0.8, alpha=0.8)

plt.show()

In [ ]:
# Simulate with APF
from swarm_prm.solvers.micro.apf import APFWaypointSolver

apf_solver = APFWaypointSolver(gaussian_prm.obstacle_map, single_agent_paths, agent_radius,
                                attract_coeff=1,
                                agent_repel_coeff=0.2,
                                repel_coeff=0.5,
                                obs_thresh=1,
                                step_size=0.5,
                                reach_dist=5,
                                max_timestep_iter=1000
                               )
paths = apf_solver.get_solution()

In [ ]:
fig, ax = gaussian_prm.visualize_map()
for i, path in enumerate(paths):
    x_coords = [loc[0] for loc in path]
    y_coords = [loc[1] for loc in path]
    ax.plot(x_coords, y_coords, '-', label='Path', color=colors[i], linewidth=0.8, alpha=0.8)

plt.show()

In [ ]:
# Animate path
from IPython.display import Image

fig_path = "solutions"
fig, ax = gaussian_prm.visualize_map()
apf_solver.animate_solution(paths, fig, ax, fig_path=fig_path)
Image(filename="solutions/apf_solution.gif")

